# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [52]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [17]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2466497635261,
 'week52high': 153.13,
 'week52low': 103.9,
 'week52highSplitAdjustOnly': 150,
 'week52lowSplitAdjustOnly': 108.1,
 'week52change': 0.3130248781109433,
 'sharesOutstanding': 17336855856,
 'float': 0,
 'avg10Volume': 58352972,
 'avg30Volume': 82476483,
 'day200MovingAvg': 137.86,
 'day50MovingAvg': 145.18,
 'employees': 153406,
 'ttmEPS': 5.2,
 'ttmDividendRate': 0.8762797928881078,
 'dividendYield': 0.00586798163802448,
 'nextDividendDate': '',
 'exDividendDate': '2021-07-27',
 'nextEarningsDate': '2021-10-14',
 'peRatio': 28.327284138564128,
 'beta': 1.5633672218721837,
 'maxChangePercent': 56.520540948608684,
 'year5ChangePercent': 4.768586475588279,
 'year2ChangePercent': 2.007422383342317,
 'year1ChangePercent': 0.32625989979760794,
 'ytdChangePercent': 0.11007784547480039,
 'month6ChangePercent': 0.0807454130172545,
 'month3ChangePercent': 0.12916194836082504,
 'month1ChangePercent': 0.008319285979332784,
 'day30ChangePerce

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [20]:
data['peRatio']

28.327284138564128

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [26]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
# print(symbol_strings[i])

# for symbol_string in symbol_strings :
#     print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'peRatio', 'Number of Shares to Buy']
print(my_columns)

['Ticker', 'Price', 'One-Year Price Return', 'peRatio', 'Number of Shares to Buy']


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [61]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#     print(data)
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   data[symbol]['stats']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
final_dataframe

,Ticker,Price,One-Year Price Return,peRatio,Number of Shares to Buy
0,A,158.75,0.624933,52.512765,N/A
1,AAL,22.06,0.592876,-2.30972,N/A
2,AAP,210.22,0.376319,21.518087,N/A
3,AAPL,150.00,0.336985,28.545351,N/A
4,ABBV,119.25,0.294967,31.100703,N/A
...,...,...,...,...,...
500,YUM,134.55,0.497359,30.218852,N/A
501,ZBH,149.53,0.066169,35.239804,N/A
502,ZBRA,578.00,1.031006,41.055437,N/A
503,ZION,55.98,0.672223,5.594326,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [62]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe.sort_values('peRatio', ascending = True, inplace = True)
final_dataframe

<ipython-input-62-f32b3729e3a7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe.sort_values('peRatio', ascending = True, inplace = True)


,Ticker,Price,One-Year Price Return,peRatio,Number of Shares to Buy
31,GE,106.760,1.063625,-331.7866,N/A
28,DXC,41.100,1.072665,-69.871981,N/A
23,MRO,12.520,1.09852,-18.034468,N/A
25,MGM,41.214,1.095442,-8.803507,N/A
21,COTY,8.810,1.152382,-7.573936,N/A
13,UA,22.820,1.323272,0,N/A
33,GM,55.900,1.058952,6.306669,N/A
4,COF,176.250,1.616937,6.535016,N/A
38,GS,418.110,1.000125,7.112271,N/A
5,DFS,136.280,1.605953,8.113145,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [63]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000
10000


In [64]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\Martin\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,peRatio,Number of Shares to Buy
31,GE,106.760,1.063625,-331.7866,1
28,DXC,41.100,1.072665,-69.871981,4
23,MRO,12.520,1.09852,-18.034468,15
25,MGM,41.214,1.095442,-8.803507,4
21,COTY,8.810,1.152382,-7.573936,22
13,UA,22.820,1.323272,0,8
33,GM,55.900,1.058952,6.306669,3
4,COF,176.250,1.616937,6.535016,1
38,GS,418.110,1.000125,7.112271,0
5,DFS,136.280,1.605953,8.113145,1


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [67]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.replace(to_replace=[None], value=0, inplace=True)
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [68]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
#         hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

# # Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_dataframe[percentile_col])

# #Print the entire DataFrame    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,157.430,N/A,0.611521,75.247525,0.279522,80.594059,0.172259,90.49505,0.044584,74.653465,N/A
1,AAL,21.172,N/A,0.577925,70.29703,0.186536,62.376238,-0.065857,28.910891,-0.016284,32.277228,N/A
2,AAP,212.010,N/A,0.359832,47.326733,0.293278,81.386139,0.005396,52.277228,-0.042239,20.990099,N/A
3,AAPL,146.100,N/A,0.325703,41.782178,0.078120,31.287129,0.124869,82.574257,0.008315,49.306931,N/A
4,ABBV,115.920,N/A,0.294908,37.623762,0.108103,40.19802,-0.003702,47.524752,-0.010714,35.049505,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,139.030,N/A,0.516808,65.544554,0.316070,83.762376,0.107464,79.80198,0.126835,96.039604,N/A
501,ZBH,150.610,N/A,0.066119,16.237624,-0.071416,6.138614,-0.147178,7.326733,-0.075391,10.29703,N/A
502,ZBRA,587.500,N/A,1.006001,92.673267,0.331159,86.138614,0.159079,87.722772,0.030577,65.346535,N/A
503,ZION,55.970,N/A,0.677759,80.0,0.114294,41.980198,-0.075172,24.950495,0.044053,73.861386,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [69]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [70]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [71]:
portfolio_input()

Enter the value of your portfolio:
That's not a number! 
 Try again:
Enter the value of your portfolio:10000


In [72]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\Martin\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,157.430,1,0.611521,75.247525,0.279522,80.594059,0.172259,90.49505,0.044584,74.653465,80.247525
1,AAL,21.172,9,0.577925,70.29703,0.186536,62.376238,-0.065857,28.910891,-0.016284,32.277228,48.465347
2,AAP,212.010,0,0.359832,47.326733,0.293278,81.386139,0.005396,52.277228,-0.042239,20.990099,50.49505
3,AAPL,146.100,1,0.325703,41.782178,0.078120,31.287129,0.124869,82.574257,0.008315,49.306931,51.237624
4,ABBV,115.920,1,0.294908,37.623762,0.108103,40.19802,-0.003702,47.524752,-0.010714,35.049505,40.09901
5,ABC,123.300,1,0.205020,27.326733,0.128291,46.732673,-0.001546,48.712871,0.044469,74.257426,49.257426
6,ABMD,350.570,0,0.115667,19.009901,0.070245,27.524752,0.170167,89.90099,0.069588,85.742574,55.544554
7,ABT,124.680,1,0.235551,30.891089,-0.008860,13.861386,0.039678,62.772277,0.033000,66.336634,43.465347
8,ACN,322.344,0,0.417473,55.049505,0.253596,76.435644,0.101548,79.009901,0.025947,61.782178,68.069307
9,ADBE,635.200,0,0.405635,53.861386,0.273650,80.0,0.297512,97.227723,0.042081,73.069307,76.039604


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [73]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [74]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [75]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [76]:
writer.save()